Transform matrixes:
* eular matrix, $T(\alpha,\beta,\gamma)$
* axis-angle matrix, $T(\textbf{r},\theta)$
* quaternion matrix, $T(x,y,z,w)$ 

Those matrixes have different parameters but same values.

In [1]:
from sympy import *
def rx(s):
    x = symbols(s)
    m = Matrix([[1,0,0],[0,cos(x),-sin(x)],[0,sin(x),cos(x)]])
    return m
def ry(s):
    y=symbols(s)
    m = Matrix([[cos(y),0,sin(y)],[0,1,0],[-sin(y),0,cos(y)]])
    return m
def rz(z):
    z=symbols(z)
    m = Matrix([[cos(z),-sin(z),0],[sin(z),cos(z),0],[0,0,1]])
    return m
rz('alpha')*ry('beta')*rx('gamma')

Matrix([
[cos(alpha)*cos(beta), -sin(alpha)*cos(gamma) + sin(beta)*sin(gamma)*cos(alpha), sin(alpha)*sin(gamma) + sin(beta)*cos(alpha)*cos(gamma)],
[sin(alpha)*cos(beta),  sin(alpha)*sin(beta)*sin(gamma) + cos(alpha)*cos(gamma), sin(alpha)*sin(beta)*cos(gamma) - sin(gamma)*cos(alpha)],
[          -sin(beta),                                     sin(gamma)*cos(beta),                                    cos(beta)*cos(gamma)]])

In [2]:
from sympy import *
init_printing(use_unicode=True)
sx = symbols('s_x')
sy = symbols('s_y')
sz = symbols('s_z')
s=Matrix([
    [sx,0,0,0],
    [0,sy,0,0],
    [0,0,sz,0],
    [0,0,0,1]
         ])
s

⎡sₓ   0    0   0⎤
⎢               ⎥
⎢0   s_y   0   0⎥
⎢               ⎥
⎢0    0   s_z  0⎥
⎢               ⎥
⎣0    0    0   1⎦

# Rotation

## Definition
**Rotation.matrix** stores 4X4 rotation matrix.

In [6]:
Rotation().matrix

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]], dtype=torch.float64)

## Copy
**Rotation.clone()** will return a deep copy of original rotation

In [1]:
from scenario import *
r=Rotation()
c=r.clone()
print(id(r),id(c))
print(r==c)
r.matrix==c.matrix

140357054280336 140357054256464
True


tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

## Compare

In [8]:
a=Rotation.Eular(0,0.1,0.3)
b=Rotation.Axis_angle(Vector3(1,1,1),2)
a*b==b*a
print((a*b).matrix)
print((b*a).matrix)

tensor([[ 0.9691, -0.2336, -0.0790,  0.0000],
        [ 0.2244,  0.9682, -0.1103,  0.0000],
        [ 0.1023,  0.0891,  0.9908,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64)
tensor([[-0.1522,  0.9570, -0.2469,  0.0000],
        [-0.3177,  0.1891,  0.9291,  0.0000],
        [ 0.9359,  0.2198,  0.2753,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  1.0000]], dtype=torch.float64)


In [10]:
c=a.clone()
c*=b
print(c==a*b)
print(c==b*a)

True
False


An object's rotation can be changed by set the its **lookat** point.

In [3]:
from scenario import *
scen=Scenario()
cube=Cube()
cube.scale=Vector3(2,1,1)
cube.lookat(Vector3(1,2,3))
cube.color=Color(r=1)
scen.add(cube)
scen.render()

Each object has a local vector to represent its face **direction**. You can get and set it directly.

In [4]:
print(cube.direction)
cube.direction=Vector3(0,1,0)
print(cube.direction)

tensor([1., 0., 0.], dtype=torch.float64)
tensor([0., 1., 0.], dtype=torch.float64)


Show the direction by a line of type "Vector"

In [5]:
d=Line.Vector(cube.direction*3)
cube.add(d)
scen.render()

# Position

In [1]:
from scenario import *
scen=Scenario()
cube=Cube()
cube.color=Color(r=1)
point=Sphere()
point.radius=0.1
point.position=Vector3(0.5,0.5,0.5)
point.color=Color(b=1)
scen.add(cube,point)
scen.render()

In [2]:
cube.lookat(Vector3(1,2,3))
cube.scale=Vector3(2,1,1)
cube.position=Vector3(0,-1,3)
point.position=cube.transform_local_position(Vector3(0.5,0.5,0.5))
scen.render()

How to convert velocity in local space to world space?\
Velocity in local space is a vector in local space.\
Velocity in world space is a vector in world space.\
**Transform a local vector to world**.

In [2]:
from scenario import *
scen=Scenario()
local=Vector3(1,1,1)
local_v=Line.Vector(local)
xyz=XYZ()
xyz.position=Vector3(1,2,3)
xyz.rotation=Rotation.Eular(0,1,-1)
xyz.add(local_v)
scen.add(xyz)
scen.render()

In [4]:
world=xyz.rotation*local
world_v=Line.Vector(world)
scen.add(world_v)
scen.render()